In [1]:
import pandas as pd
import requests

In [2]:
df = pd.read_json('/Users/eric/Downloads/bso-clinical-trials-2022-01-06_20220107.jsonl.gz', lines=True)

In [9]:
for i, row in df.iterrows():
    derived = [e for e in row.references if e['ReferenceType'] == 'derived' and 'protocol' not in e['ReferenceCitation'].lower()]
    result = [e for e in row.references if e['ReferenceType'] == 'result']
    df.at[i, 'has_publications_result_result'] = (len(result) > 0)
    df.at[i, 'has_publications_result_derived'] = (len(derived) > 0)
    df.at[i, 'has_publications_result_2'] = (len(derived) > 0) or (len(result) > 0)
    df.at[i, 'has_results_or_publications1'] = (row.has_results) or (len(result) > 0)
    df.at[i, 'has_results_or_publications2'] = (row.has_results) or (len(result) > 0) or (len(derived) > 0)

In [24]:
ct_completed = df[ (df.study_type=="Observational") & (df.study_completion_year>=2010) & (df.study_completion_year<=2020) & (df.status=='Completed') ]

In [25]:
print(ct_completed.has_results_or_publications1.mean())
print(ct_completed.has_results_or_publications2.mean())

0.08811594202898551
0.2289855072463768


In [67]:
ct_completed[ct_completed.lead_sponsor_type=="industriel"].has_results_or_publications.value_counts(dropna=False, normalize=True)

True     0.761794
False    0.238206
Name: has_results_or_publications, dtype: float64

In [6]:
sample_academique = df[(df.status=='Completed') & (df.has_results==False) & (df.study_completion_year>=2010) & (df.study_completion_year<=2020) & (df.has_publications_result_result==False) & (df.has_publications_result_derived==True) & (df.lead_sponsor_type=='academique')].sample(50, random_state=1)
sample_academique['sample'] = 'academique'
sample_industriel = df[(df.status=='Completed') & (df.has_results==False) & (df.study_completion_year>=2010) & (df.study_completion_year<=2020) & (df.has_publications_result_result==False) & (df.has_publications_result_derived==True) & (df.lead_sponsor_type=='industriel')].sample(51, random_state=1)
sample_industriel['sample'] = 'industriel'

In [7]:
sample_industriel.tail()

,all_sources,has_results,NCTId,other_ids,title,acronym,summary,study_start_date,study_start_date_type,status,...,first_results_or_publication_date,delay_first_results_completion,results_first_submit_date,results_first_submit_qc_date,WHO,ISRCTN,has_publications_result_result,has_publications_result_derived,has_publications_result_2,sample
19086,[clinical_trials],False,NCT00389909,"[{'type': 'org_study_id', 'id': 'MRAP060309', ...",Evaluation of Dosing Chart Taking Into Account...,NaN,Doxapram is used to stimulate respiration. For...,2006-11-06T00:00:00,NaN,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,industriel
21600,[clinical_trials],False,NCT02484144,"[{'type': 'org_study_id', 'id': '2014-04', 'so...",Evaluation of the Information of the Patient B...,INFOCORO,"French study, multicentrique with the cooperat...",2015-07-06T00:00:00,NaN,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,industriel
6294,[clinical_trials],False,NCT02699762,"[{'type': 'org_study_id', 'id': '2015-A01677-4...",Randomised Controlled Trial to Evaluate the Ef...,Armautotox,To show that the addition of a self-rehabilita...,2016-03-06T00:00:00,Actual,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,industriel
28645,[clinical_trials],False,NCT00349895,"[{'type': 'org_study_id', 'id': 'HEALING IIb',...",Healthy Endothelial Accelerated Lining Inhibit...,NaN,"This is a multi-center, prospective, non-rando...",2006-08-06T00:00:00,NaN,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,industriel
18217,[clinical_trials],False,NCT00903422,"[{'type': 'org_study_id', 'id': '112509', 'sou...","Study PMA112509, a Phase I/II Study of Eltromb...",NaN,This study will evaluate the safety and tolera...,2009-05-14T00:00:00,Actual,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,industriel


In [169]:
dd = pd.concat([sample_academique, sample_industriel])[['NCTId', 'sample']]
dd['url'] = dd.NCTId.apply(lambda x:"https://clinicaltrials.gov/ct2/show/"+x)
dd.to_csv('sample.csv', index=False)

In [68]:
final_data = { 'academique': {'nb_results': 0, 'nb_publications_results': 0, 'nb_publications_derived': 0, 'total': 0}}
final_data ['industriel'] = {'nb_results': 0, 'nb_publications_results': 0, 'nb_publications_derived': 0, 'total': 0}

for i, row in ct_completed.iterrows():
    if row.lead_sponsor_type != row.lead_sponsor_type:
        continue
    final_data[row.lead_sponsor_type]['total'] += 1
    if row.has_results:
        final_data[row.lead_sponsor_type]['nb_results'] += 1
    elif row.has_publications_result_result:
        final_data[row.lead_sponsor_type]['nb_publications_results'] += 1
    elif row.has_publications_result_derived:
        final_data[row.lead_sponsor_type]['nb_publications_derived'] += 1
        
final_data

{'academique': {'nb_results': 67,
  'nb_publications_results': 215,
  'nb_publications_derived': 846,
  'total': 3685},
 'industriel': {'nb_results': 3306,
  'nb_publications_results': 133,
  'nb_publications_derived': 494,
  'total': 5172}}

In [63]:
pd.DataFrame(final_data)

,academique,industriel
nb_results,67,3306
nb_publications_results,215,133
nb_publications_derived,846,494
total,3685,5172


In [42]:
( 3306 + 133 + 501 * (1 - 0.02)) / 5172

0.7598569218870843

In [40]:
(67+215+902*(1-0.16))/3685

0.28213839891451825

In [69]:
(67+215+846)/3685

0.30610583446404344

In [53]:
ct_completed[ct_completed.lead_sponsor.apply(lambda x:'rennes' in str(x).lower() )][['NCTId', 'eudraCT', 'lead_sponsor', 'has_results', 'has_publications_result_result', 'has_publications_result_derived']].has_publications_result_derived.value_counts()

False    77
True      9
Name: has_publications_result_derived, dtype: int64

In [73]:
def get_publications(nctid):
    url = f'https://clinicaltrials.gov/api/query/full_studies?expr={nctid}&fmt=json'
    r = requests.get(url).json()
    references = r['FullStudiesResponse']['FullStudies'][0]['Study']['ProtocolSection']['ReferencesModule']['ReferenceList']['Reference']
    results, derived = [], []
    for e in references:
        if e['ReferenceType'] == 'result':
            if e.get('ReferencePMID'):
                results.append(e.get('ReferencePMID'))
            elif e.get('ReferenceCitation'):
                results.append(e.get('ReferenceCitation'))
            else:
                results.append('missing')
        
        if e['ReferenceType'] == 'derived':
            if e.get('ReferencePMID'):
                derived.append(e.get('ReferencePMID'))
            elif e.get('ReferenceCitation'):
                derived.append(e.get('ReferenceCitation'))
            else:
                derived.append('missing')

    return {'nctid': nctid, 'results': results, 'derived': derived}

In [77]:
data = []
ix = 0
for nct in sample_industriel.NCTId.tolist():
    elt = {'lead_sponsor_type': 'industriel'}
    elt.update(get_publications(nct))
    data.append(elt)
    ix += 1
    if ix % 5 == 0:
        print(ix, end=',')
for nct in sample_academique.NCTId.tolist():
    elt = {'lead_sponsor_type': 'academique'}
    elt.update(get_publications(nct))
    data.append(elt)
    ix += 1
    if ix % 5 == 0:
        print(ix, end=',')

5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,

In [78]:
df = pd.DataFrame(data)

In [83]:
df['nb_derived'] = df.derived.apply(lambda x:len(x))
df['nb_results'] = df.results.apply(lambda x:len(x))

In [84]:
df.nb_derived.

,lead_sponsor_type,nctid,results,derived,nb_derived,nb_results
0,industriel,NCT01179048,"[27295427, 26855018, 26864124, 27320184, 27274...","[34704120, 33504496, 32744418, 32647053, 32643...",14,23
1,industriel,NCT01808287,[27290682],[25456759],1,1
2,industriel,NCT01475838,"[24908551, 24908550]","[28555519, 26286337]",2,2
3,industriel,NCT01874431,"[26325557, 31583611, 33107592]",[33358451],1,3
4,industriel,NCT02788006,[32334940],[],0,1
...,...,...,...,...,...,...
95,academique,NCT01710579,[28251732],[],0,1
96,academique,NCT04858204,"[1. Malhotra J, Jabbour SK, Aisner J. Current ...",[],0,1
97,academique,NCT02268708,"[17765528, 21145719]",[],0,2
98,academique,NCT03730584,[32939751],[],0,1
